In [90]:
import tensorflow as tf
from tensorflow import keras

In [91]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

## 层解读
输入2，输出 4
其中的一个样本：[x1, x2]

层计算的输出过程
xw + b = y （这里面 x，w，b，y 都是向量）

w11x1 + w21x2 + b1 = y1
w12x1 + w22x2 + b2 = y2
w13x1 + w23x2 + b3 = y3
w14x1 + w24x2 + b4 = y4

参数矩阵 2 * 4
w = 
w11  w12  w13  w14
w21  w22  w23  w24

x = 【x1  x2】

样本数是 2，输出是 2 * 4

In [92]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.01134215  0.02492263 -0.06186645 -0.00021622]
 [ 0.01134215  0.02492263 -0.06186645 -0.00021622]], shape=(2, 4), dtype=float32)


## 层中可以嵌套，可递归组合

In [93]:
## 定义新的线性层
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        print('Linear __init__:' + str(units))
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        print('Linear build:' + str(input_shape))
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        print('Linear call:' + str(inputs))
        return tf.matmul(inputs, self.w) + self.b

In [94]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        print('MLPBlock __init__')
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        print('MLPBlock call: ' + str(inputs.shape))
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

## 层的输入的思考
输入是 3, 64
模型是 64, 32, 32, 1
模型相乘：矩阵 (3, 64)与 (64, 32)相乘，结果形状是 3,32
思考：
   假如输入是(64,), 64个特征，样本数量未知，结果是(32,)也就是最后产生的是：none * 32个数据
现在输入是(3, 64), 理解成 3 个特征，每个特征 64位。也可以理解成 3 组的 64个特征。最后产生 none * 3 * 32个数据

In [95]:
mlp = MLPBlock()
one = tf.ones(shape=(3, 64)) # 3个特征，每个特征是 64位
mlp_model = tf.keras.Sequential([mlp])
y = mlp_model(one)  # The first call to the `mlp` will create the weights

MLPBlock __init__
Linear __init__:32
Linear __init__:32
Linear __init__:1
MLPBlock call: (3, 64)
Linear build:(3, 64)
Linear call:Tensor("Placeholder:0", shape=(3, 64), dtype=float32)
Linear build:(3, 32)
Linear call:Tensor("mlp_block_15/Relu:0", shape=(3, 32), dtype=float32)
Linear build:(3, 32)
Linear call:Tensor("mlp_block_15/Relu_1:0", shape=(3, 32), dtype=float32)
MLPBlock call: (3, 64)
Linear call:tf.Tensor(
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [96]:
# 这里我们能发现，这个层的接口，中间嵌套了一些 layer，这个 model 是追踪不到的
mlp_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mlp_block_15 (MLPBlock)     (3, 1)                    3169      
                                                                 
Total params: 3169 (12.38 KB)
Trainable params: 3169 (12.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [98]:
# 我们看看权重相关细节
for weight in mlp.weights:
    print(weight.name, weight.shape)

mlp_block_15/linear_49/Variable:0 (64, 32)
mlp_block_15/linear_49/Variable:0 (32,)
mlp_block_15/linear_50/Variable:0 (32, 32)
mlp_block_15/linear_50/Variable:0 (32,)
mlp_block_15/linear_51/Variable:0 (32, 1)
mlp_block_15/linear_51/Variable:0 (1,)


## 对于层次模型的调用顺序
也就是下面的代码的思考：
### tf.one(shape=(3, 64)) 与 tf.keras.Input(shape=(64,)) 与 tf.keras.Input(shape=(3, 64))
输入数据的形状：当你使用 tf.ones(shape=(3, 64)) 创建一个张量时，你得到的是一个形状为 (3, 64) 的张量，这意味着有 3 个样本，每个样本有 64 个特征。这是因为 TensorFlow（和大多数深度学习框架）通常期望数据的第一个维度是批次大小（即样本数量），剩下的维度是特征的维度。
tf.keras.Input 的用法：当你定义一个模型输入层 tf.keras.Input((64,)) 时，你告诉框架期望的输入将是一个形状为 (None, 64) 的张量，其中 None 是一个占位符，代表任意大小的批次。这意味着模型期望每个输入样本有 64 个特征，并且可以接受任意数量的样本作为批次输入。
特征数量的理解：对于 tf.keras.Input((3,64))，这会创建一个期望输入形状为 (None, 3, 64) 的模型。在这种情况下，你告诉模型每个样本实际上是一个形状为 (3, 64) 的二维数组，这可能代表一个有 3 个时间步长的序列，每个时间步长有 64 个特征，或者是其他类似的数据结构。这并不意味着有 3*64 个不同的特征，而是数据的结构更加复杂，比如时间序列数据、图像数据等。


In [101]:
mlp = MLPBlock()
input = tf.keras.Input((64,))
mlp(input)
for weight in mlp.weights:
    print(weight.name, weight.shape)

MLPBlock __init__
Linear __init__:32
Linear __init__:32
Linear __init__:1
MLPBlock call: (None, 64)
Linear build:(None, 64)
Linear call:Tensor("Placeholder:0", shape=(None, 64), dtype=float32)
Linear build:(None, 32)
Linear call:Tensor("mlp_block_18/Relu:0", shape=(None, 32), dtype=float32)
Linear build:(None, 32)
Linear call:Tensor("mlp_block_18/Relu_1:0", shape=(None, 32), dtype=float32)
mlp_block_18/linear_58/Variable:0 (64, 32)
mlp_block_18/linear_58/Variable:0 (32,)
mlp_block_18/linear_59/Variable:0 (32, 32)
mlp_block_18/linear_59/Variable:0 (32,)
mlp_block_18/linear_60/Variable:0 (32, 1)
mlp_block_18/linear_60/Variable:0 (1,)


In [102]:
mlp = MLPBlock()
input = tf.keras.Input((3, 64))
mlp(input)
for weight in mlp.weights:
    print(weight.name, weight.shape)

MLPBlock __init__
Linear __init__:32
Linear __init__:32
Linear __init__:1
MLPBlock call: (None, 3, 64)
Linear build:(None, 3, 64)
Linear call:Tensor("Placeholder:0", shape=(None, 3, 64), dtype=float32)
Linear build:(None, 3, 32)
Linear call:Tensor("mlp_block_19/Relu:0", shape=(None, 3, 32), dtype=float32)
Linear build:(None, 3, 32)
Linear call:Tensor("mlp_block_19/Relu_1:0", shape=(None, 3, 32), dtype=float32)
mlp_block_19/linear_61/Variable:0 (64, 32)
mlp_block_19/linear_61/Variable:0 (32,)
mlp_block_19/linear_62/Variable:0 (32, 32)
mlp_block_19/linear_62/Variable:0 (32,)
mlp_block_19/linear_63/Variable:0 (32, 1)
mlp_block_19/linear_63/Variable:0 (1,)
